In [11]:
import numpy as np


In [1]:
import pandas as pd
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import re


ModuleNotFoundError: No module named 'spacy'

In [2]:
!pip install spacy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 3.3 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.7/636.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 6.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 5.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 6.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [3]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.2 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import pandas as pd
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import re


In [5]:
# Load spaCy English model
import spacy
NER = spacy.load("en_core_web_sm")

# Load the 20th-century article
with open("20th_century_wiki.txt", "r", errors="ignore") as file:
    data = file.read().replace('\n', ' ')

# Apply NER to the text
doc = NER(data)


In [6]:
import pandas as pd

sentences_data = []

for sent in doc.sents:
    entities = [ent.text for ent in sent.ents]
    sentences_data.append({"sentence": sent.text, "entities": entities})

df_sentences = pd.DataFrame(sentences_data)
df_sentences.head()


,sentence,entities
0,Timeline of the 20th century - Wikipedia ...,"[the 20th century, Search Search ..."
1,1.9 1909 2 1910s Toggle 1910s su...,"[1910s, 2.1, 2.3 1912 2.4 1913 ..."
2,Further reading 15 References ...,"[References Toggle, the 20th..."
3,Color (beta)AutomaticLightDarkThis page is alw...,[Color]
4,"From Wikipedia, the free encyclopedia hide...",[Wikipedia]


In [7]:
# Example country list (replace with your own list or import from CSV)
countries = [
    "United States", "Germany", "China", "Japan", "Russia", "France",
    "United Kingdom", "Canada", "Mexico", "Italy", "India", "Brazil",
    "Argentina", "South Africa", "Turkey", "Iran", "Spain", "Australia",
    "Poland", "Iraq"
]

# Filter entities by country list
def filter_countries(entity_list, country_list):
    return [ent for ent in entity_list if ent in country_list]

df_sentences["country_entities"] = df_sentences["entities"].apply(lambda ents: filter_countries(ents, countries))
df_sentences_filtered = df_sentences[df_sentences["country_entities"].map(len) > 0]
df_sentences_filtered.head()


,sentence,entities,country_entities
17,January 22: Edward VII became King of England ...,"[January 22, Edward VII, India, Queen Victoria's]",[India]
19,June: Emily Hobhouse reports on the poor condi...,"[June, Emily Hobhouse, 45, British, Boer, Sout...",[South Africa]
21,September 7: The Eight-Nation Alliance defeats...,"[September 7, Eight, the Boxer Rebellion, China]",[China]
30,"Venezuelan crisis of 1902–1903, in which Brita...","[Venezuelan, 1902–1903, Britain, Germany, Ital...","[Germany, Italy]"
34,In Russia the Bolsheviks and the Mensheviks fo...,"[Russia, Bolsheviks, Mensheviks, the Russian S...",[Russia]


In [8]:
relationships = []

# Sliding window of 5 sentences
for i in range(df_sentences_filtered.index[-1]):
    end_i = min(i + 5, df_sentences_filtered.index[-1])
    window_countries = sum(df_sentences_filtered.loc[i:end_i, "country_entities"], [])
    unique_countries = [window_countries[i] for i in range(len(window_countries)) if i == 0 or window_countries[i] != window_countries[i-1]]
    
    if len(unique_countries) > 1:
        for idx, a in enumerate(unique_countries[:-1]):
            b = unique_countries[idx + 1]
            relationships.append({"source": a, "target": b})


In [9]:
relationship_df = pd.DataFrame(relationships)

# Sort so A-B and B-A are counted as the same
relationship_df = pd.DataFrame(
    pd.np.sort(relationship_df.values, axis=1),
    columns=["source", "target"]
)

# Count occurrences
relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source", "target"], as_index=False).sum()
relationship_df.head()


AttributeError: module 'pandas' has no attribute 'np'

In [10]:
relationship_df.to_csv("20th_century_country_relationships.csv", index=False)


In [12]:
# === Install & Import ===
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import re

# Load SpaCy English model (only needs to be done once)
# !python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

# === Load Text ===
with open("20th_century_wiki.txt", "r", errors="ignore") as file:
    text = file.read().replace("\n", " ")

# === Create NER Object ===
doc = nlp(text)

# === Optional: Visualize slice ===
# displacy.render(doc[300:1000], style="ent", jupyter=True)

# === Split Into Sentences and Capture Entities ===
sentences_data = []
for sent in doc.sents:
    entity_list = [ent.text for ent in sent.ents]
    sentences_data.append({"sentence": sent.text, "entities": entity_list})

df_sentences = pd.DataFrame(sentences_data)

# === Load Country List ===
# Replace with your list of countries, or load from CSV if you saved it
countries = [
    "United States", "China", "United Kingdom", "Germany", "France", "Japan", "Russia",
    "India", "Italy", "Canada", "Australia", "Brazil", "Spain", "Poland", "South Africa",
    "Mexico", "Turkey", "Argentina", "Iran", "Iraq"
]

# Create aliases for matching — use just last word for SpaCy match
country_aliases = [country.split()[-1] for country in countries]

# === Filter Entities to Match Country List ===
def filter_entity(ent_list):
    return [ent for ent in ent_list if ent in country_aliases]

df_sentences["country_entities"] = df_sentences["entities"].apply(filter_entity)

# === Filter Out Sentences With No Country Entities ===
df_sentences_filtered = df_sentences[df_sentences["country_entities"].map(len) > 0]

# === Simplify Names to Last Word (e.g., 'United States' -> 'States') ===
df_sentences_filtered["country_entities"] = df_sentences_filtered["country_entities"].apply(
    lambda x: [ent.split()[-1] for ent in x]
)

# === Create Relationships ===
relationships = []
for i in range(df_sentences_filtered.index[-1]):
    end_i = min(i + 5, df_sentences_filtered.index[-1])
    char_list = sum((df_sentences_filtered.loc[i:end_i].country_entities), [])

    # Remove consecutive duplicates
    char_unique = [char_list[i] for i in range(len(char_list)) if i == 0 or char_list[i] != char_list[i - 1]]

    if len(char_unique) > 1:
        for idx, a in enumerate(char_unique[:-1]):
            b = char_unique[idx + 1]
            relationships.append({"source": a, "target": b})

# === Create & Aggregate Relationship DataFrame ===
relationship_df = pd.DataFrame(relationships)
relationship_df = pd.DataFrame(
    np.sort(relationship_df.values, axis=1),
    columns=["source", "target"]
)

relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source", "target"], as_index=False).sum()

# === Export to CSV ===
relationship_df.to_csv("20th_century_country_relationships.csv", index=False)

# === Optional: Preview Data ===
print(relationship_df.head())


/var/folders/0g/xr0wqnt55l9gzqjgx49c_h8r0000gp/T/ipykernel_15336/690278295.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentences_filtered["country_entities"] = df_sentences_filtered["country_entities"].apply(


   source  target  value
0  Africa   China      5
1  Africa    Iran      1
2  Africa   Japan      3
3  Africa  Poland      1
4  Africa  Russia      3


In [15]:
df_sentences_filtered.loc[:, "country_entities"] = df_sentences_filtered["country_entities"].apply(
    lambda x: [ent.split()[-1] for ent in x]
)
